In [1]:
import sqlite3 as sql
import json

In [2]:
IS_JUPYTER = True
ROOT_PATH  = '/home/jovyan/project' if IS_JUPYTER else '/config/workspace/project'
DATA_PATH  = f'{ROOT_PATH}/DoveNest/informations/jsons'
DB_PATH    = f'{ROOT_PATH}/DoveNest/informations/db'

In [3]:
load_json = lambda path: json.loads(open(path, 'r').read())

In [4]:
conn   = sql.connect(f'{DB_PATH}/sale_informations.db')
cursor = conn.cursor() 

In [5]:
searches = cursor.execute('drop table discounts').fetchall()

OperationalError: no such table: discounts

In [6]:
query = cursor.execute('select * from saleinfo').fetchall()

In [7]:
query = cursor.execute('''CREATE TABLE IF NOT EXISTS discounts(
                          idx INTEGER NOT NULL,
                          appid INTEGER NOT NULL,
                          title TEXT NOT NULL,
                          percent INTEGER NOT NULL,
                          original INTEGER NOT NULL,
                          discounted INTEGER NOT NULL,
                          platform TEXT NOT NULL,
                          date TEXT NOT NULL);
                     ''')

In [8]:
def insert_table(table_name, idx, data_tuple, conn, cursor):
    
    q, data_tuple = '', list(data_tuple)[1:]
    
    data_tuple.insert(0, idx)
    json_data = load_json(f'{DATA_PATH}/{data_tuple[1]}/{data_tuple[1]}.json')    
    data_tuple.insert(2, json_data['name'])
    
    print(data_tuple)
    for idx, data in enumerate(data_tuple, 1):
        q += '?, ' if idx != len(data_tuple) else '?'
    
    query = f'INSERT INTO {table_name} VALUES ({q})'

    cursor.execute(query, data_tuple)
    conn.commit()
    

In [9]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('saleinfo',), ('sqlite_sequence',), ('discounts',)]

In [10]:
searches = cursor.execute('select * from saleinfo').fetchall()

In [11]:
for idx, search in enumerate(searches, 1):
    try: insert_table('discounts', idx, search, conn, cursor)
    except Exception as e: print(e)

[1, 648800, 'Raft', 33, 21000, 14070, 'steam', '20230112']
[2, 227300, 'Euro Truck Simulator 2', 75, 24800, 6200, 'steam', '20230112']
[3, 289070, 'Sid Meier’s Civilization® VI', 90, 65000, 6500, 'steam', '20230112']
[4, 381210, 'Dead by Daylight', 65, 21000, 7350, 'steam', '20230112']
[5, 477160, 'Human: Fall Flat', 70, 20500, 6150, 'steam', '20230112']
[6, 368260, "Marvel's Midnight Suns", 33, 67800, 45430, 'steam', '20230112']
[7, 397540, '보더랜드 3', 85, 64900, 9730, 'steam', '20230112']
[8, 270880, 'American Truck Simulator', 75, 24800, 6200, 'steam', '20230112']
[9, 1549180, 'Propnight', 75, 11000, 2750, 'steam', '20230112']
[10, 933820, 'Endzone - A World Apart', 75, 33000, 8250, 'steam', '20230112']
[11, 1016920, 'Unrailed!', 75, 19500, 4870, 'steam', '20230112']
[12, 1123770, 'Curse of the Dead Gods', 60, 19800, 7920, 'steam', '20230112']
[13, 1227890, 'Summer Memories', 30, 20500, 14350, 'steam', '20230112']
[14, 1286680, '타이니 티나의 원더랜드', 50, 65000, 32500, 'steam', '20230112']
[1

In [12]:
query = cursor.execute('select * from discounts').fetchall()

In [13]:
def backup_table():
    with conn:
        with open(f'{DB_PATH}/sale_informations.sql', 'w') as f:

            for line in conn.iterdump(): f.write('%s\n' % line)
            print(f'[INFO] 테이블 <discounts> 백업이 완료되었습니다. ')
            
backup_table()

[INFO] 테이블 <discounts> 백업이 완료되었습니다. 
